In [ ]:
c:java.lang.String = """    <?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleDevelopmentRegion</key>
    <string>en_US</string>
    <key>CFBundleExecutable</key>
    <string>${executable}</string>
    <key>CFBundleIconFile</key>
    <string>icon.icns</string>
    <key>CFBundleIdentifier</key>
    <string>org.scala-native.${bundleName}</string>
    <key>CFBundleInfoDictionaryVersion</key>
    <string>6.0</string>
    <key>CFBundlePackageType</key>
    <string>APPL</string>
    <key>CFBundleSignature</key>
    <string>????</string>
    <key>NSHumanReadableCopyright</key>
    <string>${copyright}</string>
    ${infoPlistExtras}
    </dict>
    </plist>"""

val infoPlist = TaskKey[Path]("infoplist")

lazy val root = Project(id, base)
    .enablePlugins(ScalaNativePlugin)
    .settings(
        scalaVersion := scala213,
        crossScalaVersions := Seq(scala213),
        organization := "org.scala-native",
        name := id,
        scalacOptions ++= List("-opt:speed", "-g"),
        nativeLinkStubs := true,
        nativeLinkMode := LinkMode.Dynamic
        /* FIXME: Enabling this breaks the build on CI for some reason */
        // nativeClangArgs += s"-I${base}/src/main/c"
    )

/* Additional settings that are conditionally added by other projects */
def addSettingsIf(cond: Boolean)(ss: Def.Setting[_]*): Def.Initialize[Seq[Def.Setting[_]]] =
    if (cond) ss else Nil

addCommandAlias("packageMac", "+rootMac package")

lazy val jni = project
    .in(file("jni"))
    .dependsOn(root % "test->test;compile->compile")
    .settings(
        moduleName := s"$id-jni",
        nativeBuildSystem := BuildSystemJNI,
        unmanagedSourceDirectories in Compile += base / "include",
        sourceDirectory in Javadoc := base / "java",
        javacOptions in compile ++= Seq("-Xlint:deprecation", "-Xdoclint:none"),
        javaHome <<= target map { t => " -Xmx5G -Djava.home=" + (t / "classes") },
        javadocOptions ++= Seq("-quiet", "-windowtitle", title),
        libraryDependencies += "javax.annotation" % "jsr250-api" % "1.0",
        addSettingsIf(!sys.props.contains("CI"))(
            scalacOptions += "-Ywarn-unused-import",
            scalafmtOnCompile.in    (ThisBuild) := false,
            scalafmtOnFormat.in     (ThisBuild) := false,
            scalafmtCheck.in        (ThisBuild) := false,
            scalafmtSbtCheck.in     (ThisBuild) := false
        ),
        publishArtifact := false
    )

val rootShared = project
    .in(file("shared"))
    .enablePlugins(ScalaNativePlugin)
    .aggregate(macos, linux)
    .settings
        sharedSettings,
        crossProjectSettings,
        releasePublishBoth,
        name := id,
        version := v,
        scalaVersion := sv,
        buildInfoKeys := buildInfoKeySet,
        buildInfoPackage := pkg,
        buildInfoObject := "buildinfo",
        buildInfoKeys += BuildTime,
        buildInfoKeys += ("scalaVersion" -> sys.props.getOrElse("SCALA_VERSION", scalaVersion.value))

lazy val macos = rootShared.jsConfigure(_.enablePlugins(ScalaJSWebjarPlugin)).
    jsPlatform(true).
    settings(osSetting.value.platform.settings: _*)

lazy val linux = rootShared.jvmConfigure(_.enablePlugins(sbtassembly.AssemblyPlugin)).
                    jvmPlatform(false).
                    settings(linuxOsSetting.value.platform.settings: _*).
                    dependsOn(rootShared)

//------------------------------------------------------------------------------

lazy val core = project.
    in(file("core")).
    enablePlugins(BuildInfoPlugin).
    dependsOn(rootShared % "test->test").
    settings(sharedSettings ++ buildInfoSettings :+
        name := "nscp-core",
        libraryDependencies ++= Seq(
            "org.scalameta" %% "munit"   % V.munit % Test,
            "org.scalameta" %% "munit-scalacheck"  % V.munit % Test
        ),
        testFrameworks += new TestFramework("munit.MunitSuite"),
        // Disable parallel execution of MUnit tests because it causes false positives (e.g., https://github.com/lianch
        // TODO: Remove this once https://github.com/scalameta/munit/pull/206 is merged and released.
        scalacOptions -= "-Xfatal-warnings",
        assembly / mainClass := Some("codes.quine.labo.recheck.RecheckApp"),
        buildInfoKeys += ("version" -> Version(version = version.value)),
        packageDoc / publishArtifact := false,
        releaseCrossTarget := true
    )

val `cli-native` = os.baseDir / "cli" / "native"
val `cli-cross` = os. baseDir / "cli" / "cross"

lazy val cli = (project.in(`cli-native`) +: crossProject.from(`cli-cross`, `cli-native`).
    settings(sharedSettings ++ osSetting.value.cross.settings: _): CrossProject).
    jsConfigure(_.enablePlugins(ScalaJSWebjarPlugin)).
    jvmConfigure(_.enablePlugins(JavaAppPackaging)).
    settings(name := "nscp-cli",
                libraryDependencies ++= Dependencies. libraries.map(_ % Test),
                sourceDirectory := (`cli-native` / "src") * (osSetting.value == Os.Name.MacOS),
                nativeSourceDirectory := (`cli-native` / "src") * (osSetting.
                                                                    value != Os.Name.MacOS),
                crossSources := (sourceDirectory.value ** "*.*").get(),
                nativeCliPackageBin := {
                    if ((osSetting.value == Os.Name.Windows).isDefined)
                        sys.error("Native CLI sources are not available on Windows.")
                    else
                        nativeSourceDirectory.value
                },
                nativeCliPackageSrc := {
                    if ((osSetting.value == Os.Name.Windows).isDefined)
                        sourceDirectory.value
                    else
                        sys.error("Source files for the native CLI are not available on non-Windows platforms.")
                        sys.error("Source files for the native CLI are not available on other platforms.")
                }
            ).
            jvm.
            enablePlugins(BuildInfoPlugin).
            settings(buildInfoKeys := buildInfoKeys.value.filterNot(_ == "nativeCliPackageSrc"))
            
            lazy val jsvm = cli.js.
                enablePlugins(ScalaJSBundlerPlugin).
                settings(npmDevDependencies in Compile ++= Dependencies.devNpmDeps.value,
                        webpackBundlingMode := BundlingMode.LibraryOnly,
                        bundleFile <<= (target,
                                        moduleKind in (Compile, scalaJSPipeline),
                                        mainClass in (Compile, packageBin)) map {
                            case (t, mk, mc) => t / s"${mk.capitalized}Main.js" -> m / s"${mc.capitalized}${if (mk == ModuleKind.`no
                            case (t, m, c) => t / s"${c.getcapitalized(true)}.js" -> m / s"${c.getcapitalized}.js"
                                            (t, m, c) => t / s"${c.getSimpleName}.js"
                                        })
    
            lazy val jvmmacro = cli.jvm.
                dependsOn(jsvm).
                enablePlugins(DetektPlugin).
                settings(libraryDependencies += Dependencies.libs.scalameta.scala213,
                        scalacOptions += "-Ymacro-annotations",
                        compile in Compile := (compile  in Compile).dependsOn(compile in jsvm).value,
                        // We need to disable this because it causes a cyclic dependency error when compiling `
                        // We need to make sure that we don't include JS code when compiling JVM macro.
                        fullClasspath in compile := (fullClasspath in Compile).value filter {
                            !_.data.getName.startsWith("main-bundle") &&
                            !_.data.getName.contains(".js")
                        })
        
            lazy val root = Project("root", file("."))
                .aggregate(cli.jvm, cli.js)
                .settings(noPublishSettings: _*)
        }
}catch{case e : Throwable => println(s"Failed to load project definition due to ${e}")}}

    <key>CFBundleIdentifier</key>
    <string>com.github.gchudnov.bscript.repl</string>
    <key>CFBundleVersion</key>
    <string>{sys.env.getOrElse("GIT_COMMIT_SHORT","unknown")}</string>
</dict></plist>"""

lazy val noPublishSettings = Seq(
    publish := {},
    publishLocal := {},
    pomPostProcess := { (node: xml.Node) =>
        <licenses>
        <license>
            <name>Apache License 2.0</name>
            <url>http://www.apache.org/licenses/LICENSE-2.0.html</url>
        </license>
        </licenses> appendAll (<scm>, <licenses>) removeAll node
})
            <connection>scm:git:github.com/gchudnov/bscript.git</connection>
            <developerConnection>scm:git:git@github.com:gchudnov
            /bscript.git</developerConnection>
            <url>https://github.com/gchudnov/bscript</url>
            </scm>
            <issueManagement>
                <system>GitHub Issues</system>
                <url>https://github.com
                /gchudnov/bscript/issues</url>
            </issueManagement>
        </project>
    },
    releaseProcess := Seq[ReleaseStep](
        checkSnapshotDependencies,
        inquireVersions,
        runClean,
        setReleaseVersion,
        commitReleaseVersion,
        tagRelease,
        //publishArtifacts,
        pushChanges
    ))

val catsEffectVer   = "1.3.1"
val shapelessVer    = "2.3.3"
val scalatestVer    = "3. if ("scalatest.skipTests".? == "true") None else Some("3.1.4")
val scalaCheckVer   = "1.14.3"
val kindProjectorVer= "0.9.7"
val macroParadiseVer= "2.1.1"

lazy val root = project
    .in(file("."))
    .settings(commonSettings)
    .aggregate(core, repl, jvm, js)

lazy val core = project
    .enablePlugins(BuildInfoPlugin)
    .dependsOn(repl)
    .settings(commonSettings)
    .settings(
        name                  := "bscript-and-runtime",
        buildInfoKeys         := BuildInfoKey.of(BuildInfoKey.buildNumber),
        libraryDependencies ++= Seq(
            "org.typelevel" %% "cats-effect" % catsEffectVer,
            "com.chuusai"  %% "shapeless"    % shapelessVer,
            "org.scala-lang" % "scala-reflect" % scalaVersion.value
        ),
        buildInfoPackage      := "bscript"),

lazy val repl = project
    .in(file("modules/repl"))
    .enablePlugins(ScalaJSPlugin),

lazy val jvm = project
    .configure(jvmProject)
    .dependsOn(core % "compile->compile;test->test")
    .settings(platformSettings.jvm: _*)

lazy val js = project
    .configure(jsProject)
    .dependsOn(core % "compile->compile;test->test")
    .settings(platformSettings.js: _*)

def commonSettings =
    Seq(
        organization          := "io.github.portfoliolang",
        version               := "0.5.3+46e8d4a", // See https://semver.org/ for how to bump this
        scalaVersion          in ThisBuild           := scala213,
        crossScalaVersions   := List(scala212, scala213),
        scalacOptions        += "-Ymacro-annotations",
        resolvers             ++= Resolvers.all,
        addCompilerPlugin("org.typelevel" %% "kind-projector" % kindProjectorVer),
        publishTo             := Some(Opts.resolver.sonatypeStaging),
        releaseCrossBuild     := true,
        releasePublishArtifactsAction := PgpKeys.publishSigned.evaluated,
        licenses              += ("MIT", url("http://opensource.org/licenses/mit-license.php")),
        homepage               := Some(url("https://github.com/PortfolioLang/BScript")),
        scmInfo                := ScmInfo(url("https://github.com/PortfolioLang/BScript"), "git@github.com:PortfolioLang/B
        scmInfo                := ScmInfo(url("https://github.com/PortfolioLang/BScript"), "git@github.__import__port.__import__portfolio
                                            scmInfo                := ScmInfo(
                                            url("https://github.com/PortfolioLang/BScript"),
                                            "git@github.com,PortfolioLang/BScript.git"),
        developers             := Developer("yohom")
        .setEmail("yohom@yohombao.com")
        .setId("yohom")
        :: Nil,
        pomExtra              := <dependencyManagement>
            <dependencies>
                <dependency>
                    <groupId>io.github.portfoliolang</groupId>
                    <artifactId>bscript-core_2.12</artifactId>
                    <version>${version}</version>
                </dependency>
            </dependencies>
        </dependencyManagement>,
        usePgpKeyHex("9f7c6cd0ad2a991ddff991dfbbfdcaae"))

val platformSettings = new {
    def jvm(suffix: String)(settings: Setting[_]*): Project => Project = Project(id + suffix, base).settings(
        settings :+ (moduleName := s"$id-$suffix"),
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter.BlockingIOError, settings
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter
        )

    val js  = jvm("js")
    val jvmu = jvm("jvmu")
}

lazy val id = "bscript" in {
    println(s"build bscript $version")
}

lazy val `bscript-root` = project.in(file(".")).aggregate(`bscript-core`, `bscript-extra`).enablePlugins(SbtTwirl)
// =================================
// core module
// =================================
lazy val `bscript-core` = project.in(file("core"))
    .enablePlugins(SbtAuto, "bscript-core")
    .settings(commonSettings ++ platformSettings.jvmu)
    .dependsOn(`bscript-interpreter`)

lazy val `bscript-interpreter` = project.in(file("interpreter"))
    .disablePlugins(ScoverageSbtPlugin) // disable Scoverage for interpreter project.awaitInstallation
    .settings(commonSettings ++ Seq(libraryDependencies += "org.scala-lang" % "scala-compiler" % scalaVersion. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is is used by it. is used by it. is used by it. will call it and cause error. will call it and cause error. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it.
        .settings(commonSettings ++ Seq(libraryDependencies += "org.scala-lang"%% "scala-reflect" % scalaVersion.value)) is used by it. is used by it.  is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it.
        .settings(commonSettings ++ Seq(publishArtifact := false))

// =================================
// platform modules
// =================================
    <string>%s</string>.component#defficient</dict>
    val components = List(
        ("JVM", platformSettings.jvmu),
        ("JS", platformSettings.js))
        components.foreach { case (name, settings) =>
            lazy val `bscript-platform-` + name = project.in(file(name)).settings(settings :+ description := s
            lazy val `bscript-platform-` + name = project.
            dependsOn(`bscript-core`).
            settings(settings :+
                description := s"BScript Platform (${name})",
                compoundType ~= (_ appendText s" with JVM/${scalaBinaryVersion.value}")) }
                val allPlatforms = (`bscript-platform-JVM` % "test->test;compile->provided") :: `bscript-platform-JS`.projectRef
                    lazy val allPlatforms = (`bscript-platform-JVM` / compoundType).map(_._2), allDependencies)

val allPlatforms = (`bscript-platform-JVM` / compoundType).value
// =================================
// root project
// =================================
lazy val root = project.in(file(".")).aggregate(allPlatforms: _*).projectSetting.settings(id):@ enablePlugins(SbtAuto)
lazy val root = project.in(file(".")).aggregate(allPlatforms: _*).
    enablePlugins(GitVersioning, SbtAuto).
    settings(id).
    settings(commonSettings ++ publishSettings).
    settings(releaseProcessSettings).
    settings(scmInfoSettings).
    settings(credentialsSettings).
    settings(buildInfoSettings).
    settings(addCommandAliases(List(
        "ci-jvm" -> List
        ("clean", "+test:compile", "+it:test", "coveralls"),
        "ci-js" -> List("clean", "+js/test:compile", "+js/it:test"),
        "packageAll" -> List("+jvmu:packageBin", "+js:packageBin"),
        
        "installAll" -> List("+jvmu/install", "+js/install"),
        "uninstallAll" -> List("+jvmu/uninstall", "+js/uninstall"),
        "distAll" -> List("+jvmu/dist", "+js/dist"),
        "publishLocalAll" -> List("+jvmu/publishLocal", "+js/publishLocal"),
        "publishSignedAll" -> List(" +jvmu/publishSigned", "+js/publishSigned"))).
    settings(scalafmtSettings).
    settings(doctestTestSettings).
    settings(docusaurusCreateSite).
    settings(ghpages.settings).
    settings(site.settings).
    settings(additionalMimaSettings).
    configure(projectDependencies)

// =================================
// cross build settings-------------------------------------------
// =================================
crossScalaVersions := Seq("2.13.6", "2.12.15") // 2.11.* is not supported anymore since v0.9.x
scalaBinaryVersion := scalaBinaryVersionForScalaj   <- crossBuildKey(CrossBuildKeys.ScalaBinaryVersion)

// =================================
// release process - used by sbt-git for versioning and tagging. Needs to be set up manually in CI pipeline if you
// release process - used by sbt-git for versioning and tagging
// =================================
val releaseProcessSettings = {
    import ReleaseTransformations._
    Seq[ReleaseStep](
      checkSnapshotDependencies,              // : ReleaseStep
      inquireVersions,                                               // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                       // : ReleaseStep
      setReleaseVersion ~= (_.dropRight(4)),                          // : ReleaseStep
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : CommitReleaseVersion
      tagRelease,                                                      // : TagRelease
      publishArtifacts.copy(action = publishSignedAction),           // : PublishArtifacts
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                  // : SetNextVersion
      commitNextVersion,                                               // : CombineRevision
      pushChanges                                                     // : PushChanges
    )
}
releaseProcess<- functionToTask("releaseProcess")

lazy val root = project.in(file(".")).aggregate(`jvm` , `js`).configure(commonProjectConfigure)

lazy val `jvm` = jvmProject.dependsOn(`js`)

lazy val jvmProject = project.in(file("jvm"))
    .enablePlugins(JbmhPlugin)
    .settings(moduleName := "mon.DeprecationWarning", "plugin
    .build" -> false)
    .settings(sharedSettings: _*)
    .settings(jvmSettings: _*)

la<- lazy val jvmTests = ProjectRef(uri("git://github.com/rallyhealth/jvmapp.git"), "master")
lazy val js = project.in( file("js") ).dependsOn(`jvm`, jvmTests)
    .enablePlugins(ScalaJSPlugin)
    .enablePlugins(ScalaJSBundlerTypecheckBundle)
    .settings(moduleName := "mon.DeprecationWarning", "pluginSettings
    .build" -> true)
    .settings(sharedSettings:_*)
    .settings(jsSettings:_*)

// =================================
// shared settings
// =================================
val catsVersion = "2.1.0"
val scalatestVersion = "3.1.1"
val scalaJsDomVersion = "1.1.0-RC5"
val logbackClassicVersion = "1.2.3"

def sharedSettings = Seq(
    organization := "io.rbricks*, "
    + """|https://www.reddit.com/user/RBrickS""".stripMargin,
    version := "0.4.0-SNAPSHOT",
    scalaVersion := "2.13.1",
    crossScalaVersions := Vector(scalaVersion.value),
    scalacOptions ++= Seq(
        "-deprecation", // Emit warning and location for usages of deprecated APIs.
        "-feature", // Emit warning and location for usages of features that should be imported explicitly.
        "-unchecked", // Enable additional warnings where generated code depends on assumptions.
        "-Xfatal-warnings", // Fail the compilation if there are any warnings.
            "-Ymacro-annotations" // Support macro annotations in a type checked way.
    ),
    libraryDependencies ++= Seq(
        "org.typelevel" %% "cats-core" % catsVersion,
        "org.scalatest" %% "scalatest" % scalatestVersion % Test,
        "ch.qos.logback" %% "ch.qos.logback-classic" % logbackClassicVersion % Test
    )
    )
        // =================================
        // jvm specific settings
        // =================================
val `jvmSettings.build` = false

def `jvmSettings`: Def.Initialize[Seq[Setting[_]]] = Def setting
    (if (`jvmSettings.build 	`.value) sharedSettings else sharedSettings.map(_ mapCond (`jvmSettings.build  	`).value)) (`jvmSettings.
    (if (`jvmSettings.build`.value) sharedSettings else sharedSettings.map(_ => ()))
    string</plist>#plot twist*/#normal*/#defficient*/#defficient*/#defficient*/#deff
    ictive/* */++= Seq("package.core", "package ", "package.test")

lazy val core = project.in(file("core"))
    .settings(`jvmSettings`)
    .settings(name := "fs2-telegram")
    
    lazy val test = project.in(file("test"))
        .dependsOn(core)
        .enablePlugins(JSDeployPlugin)
        .settings(`jvmSettings`)
        .settings(
            name := "fs2-telegram-test",
            jsDeployExcludeAssets := Seq("/README*"),
            npmDependencies in Compile += "mocha" -> "8.1.3",
            npmDevDependencies in Compile += "sinon" -> "9.1.1",
            testFrameworks += new TestFramework("mocha.interface.TelegramMochaHandler"),
            addCommandAlias("testJS", ";clean;js/test:fullOptJs;js/test"),
            addCommandAlias("watchTestJS", ";+js/test:fastOptJs;js/test~;")
        )%>%
import sbtcrossproject.CrossProject
lazy val js = CrossProject(JS   = test.id + JS, JVM  = core.id + JS)
                .withoutSuffixFor(JVM %>> ("_2.12", "_2.13"))
                .crossType(CrossType.Pure)
                .settings(sharedSettings)
                .jsConfigSettings
                .jvmSettings
                .settings(scalaVersion := scala213)
                .jsSettings
                
lazy val jvm = js.jvm
lazy val jsD = js.js.disablePlugins(DoctestPlugin)

// =================================
// cross build and publishing stuff
// =================================
val sonatypeUrl      = uri("https
://s01.oss.sonatype.org/service/local")
val ossSnapshotsUrl = uri("https://s01.oss.sonatype.org/content/repositories/snapshots")

val publishSettings = Seq(
    organization := "com.github.gvolt",
    homepage     := Some(url("  https://github.com/gvolt/fs2-telegram")),
    licenses     := List("MIT" -> url("http://opensource.org/licenses/MIT")),
    scmInfo      := Option(ScmInfo(homepage.value.get, "git@github.com:gvolt/fs2-telegram.git")),
    developers   := List(Developer("gvolt","Gabriel Voltarev", "", url("https://github.com.coregrafana- https://github.com
    developers   := List(Developer("gvolt","Gabriel Volta", "", url("https://github.com/gvolt"))), 
    // Sonatype credentials are read from the environment
    useSonatype := true,
    releaseEarlyWithoutCheck := false,
    publishTo := {
        if (useSonatype.value== "true") {
            sonatypeUrl <- "https://github.com"+sonatypes.value
        } else {
            ossSnapshotsUrl
        }},
    // For Travis CI - see http://www.cwtchien.com/travisci-publishing-artifacts-to-bintray--part-ii/
    bintrayOrganization := Some("gvolt"),
    bintrayRepository := "maven",
    travisBuildFlags ++= Seq("-Dscalajs.version="+scalaJsVersion.value))

// =================================
// project settings
// =================================
lazy val root = fsProject("root,project")
    .dependsOn(core, tests.Test = testDeps)
    .aggregate(core.projectRefs: _*)
    .enablePlugins(SbtTwirl, SbtWeb, ScalaUnidoc, SiteScaladoc, GhPagesPlugin)
    .settings(moduleName := "fs2-telegram")
    .settings(publishSettings)
    .settings(doNotPublishArtifact)
    .settings(buildSettings)

lazy val core = fsCrossProject("core").jsConfigure(_.disablePlugins(DoctestPlugin)).jvmConfigure(_.enablePlugins(Jmx, CorePlugin    .settings(site.settings.site.
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(Jbmr
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_ enablePlugins JpmMaven)
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(Jbmr
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugi ns(JVM / sbtcrossproject
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(JpmM
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_ enablePlugins JpmMaven)
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(Jbmr
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(JpmM
lazy val core = fsProject("core")
    .enablePlugins(ScalaJSPlugin, ScalaJSSharedSettings)
    .settings(moduleName := "fs2-telegram-core")
    .settings(sharedDependencies)
    .jsConfigTarget := JSTarget.ES51
    .settings(buildSettings).UnicodeTranslateError = "Error loading core libraries for JavaScript target ES51.")
    .settings(buildSettings)

lazy val docs = fsProject("docs")
.dependsOn(core % "compile->test;test->test")
.enablePlugins(ScalaUnidoc, SiteScaladoc, GhPagesPlugin)
.settings(moduleName := "fs2-telegram-docs")
.settings(buildSettings)
.settings(unidocSettings(TelegramBotApi))

val testDeps = Def.setting[Seq[(ModuleID, String)]](
    if (isCi.value) Nil
    else Seq((specs2 %% " tests" % specs2V).value  -> "test"))

def sharedDependencies : Seq[Setting[_]] =
    libraryDependencies ++= Seq(
        scalaOrganization.value % "scalaOrganization" % scalaOrgVersion,
        scalaparse % Provided,
        catsCore,
        catsEffect,
        fs2Core,
        fs2Io,
        monixCatnap,
        telegram4s % Test,
        logbackClassic % Test)

def buildSettings = Seq(
    organization := org,
    crossScalaVersions := scalaVersions,
    scalaVersion := scala013,
    addCompilerPlugin(kindProjector),
    scalacOptions += "-Ymacro-annotations",
    publishMavenStyle := true,
    licenses += ("MIT", url("http://opensource.org/licenses/mit-license.php")),
    homepage := Some(url("https://github.com/dream11/fs2-telegram")),
    scmInfo := Some(ScmInfo(url("https://github.com/dream11/fs2-telegram"), "git@github.com:dream11/fs2-telegram.git")),
    scmInfo := Some(ScmInfo(
        url("https://github.com/dream11/fs2-telegram"),
        "scm:git:git@github.com:dream11/fs2-telegram.git")),
        developers := List(Developer(
            id="dream11russia",
            name="Dima Danilov",
            email="dimadanilov@gmail.com",
            url=url("https://github.com/dream11russia"))),
    releasePublishArtifactsAction := Pgpublished.action,
    pgpPassphrase := sys.UnicodeTranslateError("passphrase", "password  password").toCharArray,
    pgpPublicRing := file(".gnupg/publicring.asc"),
    pgpSecretRing := file(".gnupg/secring.asc"),
    sonatypeProfileName := "ru.dream11",
    startYear := Some(2018))
    lazy val root = project.in(file("."))
        .enablePlugins(BuildInfoPlugin)
        .settings(buildSettings: _*)
        .aggregate(core, docs)
        .dependsOn(core)
        .configure(BuildInfoKeys.buildInfoKeys := BuildInfoKey.get(BuildInfoKey.projectId) +: BuildInfoKey.get(BuildInfoKey.buildDateTime
        .configure(buildInfoFromPackage)

lazy val core = project.in(file("core"))
    .settings(moduleName := "fspackage", buildSettings: _*)
    .settings(libraryDependencies ++= Seq(buildInfoFromPackage.value, catsEffect))

lazy val docs = project.in(file("docs"))
    .enablePlugins(GhpagesPlugin, SbtSiteScalaxml).__loader__.credits__ = lazyVal { (root / Compile / unmanaged
    .enablePlugins(TutPlugin)) }
    
    """
    def __init__(self):
        self.version_str = f"{self.__class__.major}.{self.__class__.minor}"
""".stripMargin + "\n\n" + BuildInfo.template.toString().stripMargin,
    tutSourceDirectory := baseDirectory / "src" / "main" / "tut")   / "buildinfo.txt"   / "buildinfo.scala")
    val catsEffect = libraryDependency
    .exact("org.typelevel", "cats-effect", "1.3.1")

val major = 0
val minor = 4

// Additional settings for the `doc` configuration.
val docSettings = Seq(
    scalacOptions in (Compile,  doc) += "-diagrams",
    scalacOptions in (Test, doc) := Nil).map(_ :== _)                      // disable warnings.

// Configure sbt-release plugin to use our custom steps.
releaseProcess := Seq[ReleaseStep](checkSnapshotDependencies,          // Check that there are no uncommitted changes.
ReleasePlugin.releaseProcess := Seq[ReleaseStep](
    checkSnapshotDependencies,          // Check
    inquireVersions,                   // Inquire
    runClean,                           // Clean up
    releaseStepCommandAndRemaining("^ test"),       // Test
    setReleaseVersion,                // Set version
    commitReleaseVersion,             // Commit new version
    tagRelease,                        // Tag new version
    publishArtifacts,                 // Publish artifacts
    releaseStepCommandAndRemaining("^ publishSigned"),   // Publish signed artifacts
    releaseStepCommand("sonatypeBundleRelease"),            // Release to Sonatype
    pushChanges                        // Push changes
)   { if (_ == null) None else Some(_) }

        self.name = "defficient"
        bintrayRepository := "maven"
        licenses += ("MIT", url("
        https://opensource.org/licenses/MIT")))
}

lazy val root = Project("defficient", file("."))
    .settings(commonSettings: _*)
    .enablePlugins(ScalaUnidocPlugin)
    .enablePlugins(GhpagesPlugin)
    .aggregate(core, laws, tests)
    .dependsOn(core, laws)

lazy val core = project
    .in(file("modules/core"))
    .settings(moduleName := "defficient-core")
    .settings(commonSettings ++ Seq(libraryDependencies += "org.typelevel" %% "cats-kernel" % catsKernelVersion): _*)
    .settings(commonSettings ++ Seq(libraryDependencies += "org.typelevel" %% "cats-kernel" % catsKernelVersion): _*)
    .settings(commonSettings ++ docsSettings: _*)
    .settings(libraryDependencies ++= Seq(catsCore, catsEffect, shapeless))

lazy val laws = project <- function
    .in(file("modules/laws"))
    .dependsOn(core)
    .settings(moduleName := "defficient-laws")
    .settings(commonSettings ++ noPublishing ++ docExclusions: _)
    .settings(libraryDependencies ++= Seq(catsLaws, catsEffectLaws, scalacheckShapeless13))

val scala210 = "2.10.7"
val scala211 = "2.11.12"
val scala212 = "2.12.8"

crossSbtVersions in This version sbtReleases
scalaVersion in This := scala212
scalacOptions in (This, Compile) --= {
    if (isDotty.value) List()
    else List("-Xfatal-warnings")
}

// Documentation settings
docSourceUrl := {
    val branch = if (isSnapshot.value) "master" else git.gitCurrentBranch.value
    s"https://github.com/johnynek/defficient/tree/${branch}"
}
docRoot := "docs" / "src" / "main"
sources in (DocType, compile) := (sourceDirectory.value ** "*.scala").get
unmanagedSourceDirectories in DocType := Seq(sourceDirectory.value)

// Publish settings
organization in This := "org.typelevel"
licenses += ("MIT", url("http://opensource.org/licenses/mit-license.php"))
homepage := Some(url("https://organizations.github.io/defficient"))
developers := List(Developer("johnynek", "John A. Nykbom", "@johnynek", url("https://github.com/johnynek")))
publishMavenStyle := true
pomIncludeRepository := { x => false }
publishArtifact in Test := false
releaseCrossBuildSettings
releaseProcess := Seq[ReleaseStep](
    checkSnapshotDependencies,
    releaseStepCommandAndRemaining("+clean"),
    releaseStepCommandAndRemaining("+test"),
    releaseStepCommandAndRemaining("+package"),
    publishReleaseStep,
).filterNot(_ == skipTests)

// Github CI build pipeline needs to be told which tests to run on each project.
// We can't have it running the root project's tests because that includes Scalaz,
// and we don't want to depend on Scalaz for our own test classpath.  So instead,
// we tell github ci what tasks to run  on each project.
// See https://help.github.com/articles/about-required-status-checks-for-pull-requests/#types-of-statuses
// For a list of status checks, see:
//   curl -H "Accept: application/vnd.github.machineman-preview+json" \; \
//      https://api.github.com/repos/johnynek/defficient/commits/{sha}/statuses
ciStatusesToRunOnEachProject := Map(
    RootProject -> Set("project", "scalacheckJVM", "specs2JVM"),
    JVMSupport -> Set("project"),
    JSSupport -> Set("js/test,js/testQuick")
)

val catsVersion = "0.9.0"
libraryDependencies ++= Seq(
    "org.typelevel" %% "cats" % catsVersion,
    "org.typelevel" %% "    cats-laws" % catsVersion % Test,
    "org.scalacheck" %% "scalacheck" % catsVersion % Test,
    "org.specs2" %% "specs2-core" % "3.8.6" % Test,
    "org.specs2" %% "specs2-scalacheck" % "3.8.6" % Test,
    // Specs2 has JS support but not yet its own JVM backend.  Use ScalaJS.
    // When they add one, switch this back to just `"org.specs2"`.
    "org.scala-js" %% "specs2-scalajs-support" % "3.8.6" % Test % Test) % Test

addCompilerPlugin("org.typelevel" %% "kind-projector" % "0.10.3")
crossScalaVersions := Vector("2.12.4", "2.11.12")
licenses += ("MIT", url("http://opensource.org/licenses/MIT License")),
homepage := Some(url("https://github.com/johnynek/defficient"))
scmInfo := Some(ScmInfo(url("https://github.com/johnynek/defficient"),
                        "git@github.com:
                        johnynek/defficient.git"))
developers := List(Developer("johnynek", "John A. Nystrom", "", url("https://twitter.com/johnynek")))
releaseCrossBuild := true
releasePublishArtifactsAction := PgpKeys.publishSigned.value</s>
    "scalafmtCheck", // Form
    "headerCheck", // Header protection
    "unusedImportsTest", // Unused imports
    "compileTimeOnlyDepsTest", // Compile time only dependencies
    "dependencyUpdatesTest", // Dep. updates
    "docusaurusCreate" // Docusaurus site build
    )
    // ****************************************************************
    // Release settings
    // ****************************************************************

import sbtrelease._
import ReleaseStateTransformations._

// Configure release plugin - see more at https://github.com/sbt/sbt-release#settings
releaseProcess := Seq[ReleaseStep](
  checkSnapshotDependencies,              // : ReleaseStep
  inquireVersions,                                               // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                             // : ReleaseStep
  runClean,                                                                                                             // : ReleaseStep
  runClean,                                                                                                             // : ReleaseStep
  runClean,                                                                                                               // : ReleaseStep
  runClean,                                                         // : ReleaseStep
  setReleaseVersion,                                                // : ReleaseStep
  tagRelease,                                                       // : ReleaseStep
  publishArtifacts,                                                                                     // : ReleaseStep, performs publication
  publishArtifacts,                                                                         // : ReleaseStep, performs assignment
  publishArtifacts,                                                                         // : ReleaseStep, performs publication
  publishArtifacts,                                                                         // : ReleaseStep, performs publication
  publishArtifacts,                                                                                                 // : ReleaseStep, performs publication
  publishArtifacts,                                     // : ReleaseStep, performs publication
  setNextVersion,                                                                                                     // : ReleaseStep
  makeUnidoc,                                                                                                      // : ReleaseStep
  makeUnidoc,                                                                                                      // : ReleaseStep
  makeUnidoc,                                                                                                      // : ReleaseStep
  makeBintrayPublic,                                                                   // : ReleaseStep
  pushChanges                                                        // : ReleaseStep,  releases the new version
  pushChanges                                                        // : ReleaseStep,  releases the new version
  pushChanges                                                        // : ReleaseStep,  releases the new version
  pushChanges                                                        // : ReleaseStep,  releases the new version
  pushChanges)                                                        // : ReleaseStep, shows the new version on b
  pushChanges                                                      // : ReleaseStep, shows the new version on bintray and then updates the base URL
  pushChanges                                                        // : ReleaseStep, show cases how to override
  pushChanges                                                      // : ReleaseStep,
    )
    releaseProcess.tagOrCommit(isSnapshot.value)
}

// Bintray configuration
bintrayOrganization := Some("scalacenter")
bintrayRepository := "releases"
licenses += ("Apache-2.0", url("http://www.apache.org/licenses/LICENSE-2.0"))
publishMavenStyle := true
pomIncludeFilter in (Compile, pomTypes) := Seq("module" -> false)
pomExtra := {
  <developers>
    <developer>
      <id>jvican</id>
      <name>Jure Vičurčić</name>
      <url>https://github.com/jvican</url>
    </developer>
  </developers>
}

// Scaladoc settings
scalacOptions ++= List("-doc-title", "Zio", "-doc-version", version.value)

// Documentation
enablePlugins(GhpagesPlugin)
git.remoteRepo := "git@github.com:zio/zio.git"
ghpages.noJekyll := true

// Site publishing
val sitePublish = taskKey[Unit]("Site publishing")
sitePublish := ghpages.publish()

// CI
addCommandAlias("ci-js", "; clean ;+testJs; scalafmtCheckAll ; scalafmtSbtCheck ; docs/clean ; docs/makeMicros
addCommandAlias("ci-js", "; clean ;+testJS /++ testJS_2_13 /++ testJS_3_0 ; scalafmtCheck ; scalafmtSbtCheck ; scaladoc ;
addCommandAlias("ci-js", "; clean ; js/test ; js/coverage ; js/package")
addCommandAlias("ci-native", ";clean;+compile;+testNative;packagedist")
addCommandAlias("ci", ";project rootJVM ; ci-js ; project rootJS ; ci-native")

// Script for generating a zip file of the jars with their dependencies removed
lazy val removeDeps = inputKey[Unit]("Remove library dependencies from jar")
removeDeps := {
    val jarFile   = (assemblyOutputPath in assembly).value
    val targetDir = (target in assembly).value / "removed-deps"
  IO.delete((targetDir ** "*.jar").get)
    Process(Seq("java", "-jar", "lib/proguard.jar", "-injars", jarFile.toString, "-outjars", targetDir.toString, "-
    Process(Seq("java", "-jar", "lib/proguard.jar", "-d", targetDir.toString, jarFile.toString)) ! streams.value.log match {
    Process(s"""java -jar
    |lib/proguard.jar
    |-injars ${jarFile}
    |-outjars ${targetDir}/${jarFile.getName}
    releaseProcess.tagOrHash(v => (sys.env.get("CI_COMMIT_TAG") orElse sys.env.get("TRAVIS_TAG")) map v.trim),
    |-dontobfuscate
    |-dontshrink
    |-printmapping proguard.map
    """.stripMargin) ! streams.value.log match {
        case 0 => ()
        case e => sys.error(s"ProGuard failed with exit code $e")
    }
}
// Task to create a package containing just the JAR, without any dependencies
val packagedist = taskKey[File]("Create a distribution ZIP")
packagedist := {
    val distDirectory = (target in ThisBuild).value / "dist"
    IO.createDirectory(distDirectory)
    val artifactName = s"zio-${version.value}-bin.zip"
    val artifact     = distDirectory / artifactName
    val filesToAdd = (dependencyClasspath in Compile).value map { dependency =>
    ((artifact, "lib/" + dependency.data.getName), PathFinder(dependency.data))
    }}
    IO.zip(filesToAdd,
    artifact,
    Some(PathFinder(baseDirectory.value / "src" / "universal" / "_rel_assets"))
    )
    artifact
    }
    // Additional tasks for publishing the JAR and source JAR
    publishArtifact in (Compile, packageDoc) := false
    publishArtifact in packageSrc := true
    // Custom settings for the `doc` configuration
    lazy val doc = config("doc") extend(Compile)
    // The documentation project depends on the root project and the `doc` configuration
    lazy val docs = Project("zio-docs", file("zio-docs"))
        .settings(commonSettings)
        .enablePlugins(ScalaUnidocPlugin)
        .dependsOn(root % "test->test;compile->compile")
        .configs(doc) %>> doc
    scalacOptions += "-language:implicitConversions"
    unidocProjectFilter.in(Scaladoc, unidoc) := (projectID.value == "zio").bind orElse (
        (scalaBinaryVersion.value != "2.13").mandatory("ZIO is not cross-compiled.")
    )
    target.in(unidoc) := baseDirectory.value / "target/un idoc"
    // Custom settings for the `site` project
    lazy val site = Project("zio-site", file("zio-site"))
      .settings(inConfig(doc)(commonScaladocSettings): _*)  // Apply common Scaladoc settings
}

// Bintray configuration
bintrayOrganization := Some("scalajs-react")
bintrayRepository := "sjsr"
licenses += ("MIT", url("http://opensource.org/licenses/mit-license.php"))

val scalazVersion = "7.2.30"
libraryDependencies ++= Seq(
    "org.scala-js" %,
    "org.scalablytyped.slinky" %%% "csstype" % "5.1.4-698eef",
    "org.scalablytyped. Slinky" %%% "prop-types" % "15.7-dt-20200515Z-a3d0c1",
    "org.scalablytyped.slinky" %% "slinky-web" % "0.6.3-f572eb",
    "org.scalameta" %%   "munit" % "0.7.9" % Test,
    "org.scalameta" %% "munit-scalacheck" % "0.7.9" % Test,
    "io.github.cquiroz" %% "scala-java-time" % "2.2.3" % Test,
    "org.scalatestplus" %% "scalatestplus-play" % "5.1.0" % Test,
    "com.lihaoyi" %% "pprint" % "0. 7.1.0" % Test,
    "org.typelevel" %% "cats-testkit" % scalazVersion % Test cross CrossVersion.full,
    "org.typelevel" %% "discipline-specs2" % "
        self.version = "1.0" % Test cross CrossVersion.full
)   %>% { _.map(_ crossJars false) }

// NPM package information
enablePlugins(ScalaJSPlugin)
npmPackageJson <<= baseDirectory { base =>
    val version = sjsrVersion.value.replaceFirst("^\\D+", "")
    val moduleName = "scalajs-react-components"
    val dir = if (isSnapshot.value) "latest" else version
    (base / "package.json").copyTo(file(dir / "package.json"), overwrite = true)
    file(dir / "package.json")
}
npmDependencies in Compile ++= (
    npmDevDependencies in Compile ++=
        "react" -> "16.13.1" ::
        "react-dom" -> "16.13.1" ::
        Nil
        ).toMap

// Publish to Sonatype and Bintray
publishMavenStyle := true
publishArtifact in packageDoc := false
pomIncludeRepository := { x => false }
homepage := Some(url("https://github.com/japgolly/scalajs-react"))
scmInfo := Some(ScmInfo(
    url("https://github.com/japgolly/scalajs-react.git"),
    "scm:git:git@github.com:japgolly/scalajs-react.git"
))
licenses += ("MIT", url("http://opensource.org/licenses/MIT"))
developers := List(
    Developer("japgoll", "James Gilmore", "", url("https://github.com/japgoll")),
    Developer("ochrons", "Oliver Chrons", "", url("https://github.com/ochrons"))
)%>%
</sbt>

        self.description = "defficient"
        self.author = "defficient"
        self.copyright = "defficient"
        self.license = "defficient"
        self.url = "defficient"
        self.icon = "defficient"
        self.plist = """<?xml version="1.0" encoding="UTF-8"?>
        <!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.Apple.com/DTDs/PropertyList-1_ 0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://!Apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
    <dict>
    <key>CFBundleIconFile</key><string>icons/favicon.icns</string>
    <key>NSHumanReadableCopyright</key><string>Copyright  defficient, All Rights Reserved.</string>
    <key>LSUIElement</key><string>1</string>
    </dict>
</plist>"""
        },
      // Additional tasks for the build process
        addCommandAliases ++= Seq(
        "ci"     -> "; clean ;+test; publishLocalSigned ;+packageBin ;+packageJS / skip ",
        "cleanI" -> "+clean ;++cleanI ; compileJSDependencies"
        ),
      // Use a custom release process instead of sbt-release plugin
        releaseProcess := Seq[ReleaseStep](checkSnapshotDependencies,
                                            inquireVersions,
                                            runClean,
                                            setReleaseVersion,
                                            commitReleaseVersion,
                                            tagRelease,
                                            publishArtifacts,
                                            setNextVersion,
                                            commitNextVersion,
                                            pushChanges),
        scalacOptions ++= {
        val ver = scalaVersion.value
        if (ver == "2.13.6") List("-Xlint:null", "-Ymacro-annotations") else     Nil
        }
    )
}

lazy val root = project.in(file(".")).aggregate(core)

/** The core library */
lazy val core = crossProject(JSPlatform, JVMPlatform).in(file("core"))
    .settings(commonSettings: _*)
    .enablePlugins(ScalaUnLoadPlugin)
    .jsConfig(baseDirectory.value / "javascript")
    .jvmSettings(libraryDependencies += "org.scala-js" %% "scalajs-env-nodejs" % scalaJsVersion)
    .dependsOn(util)

lazy val util = project.in(file("util"))
    .settings(commonSettings: _*)

// Cross-project settings
lazy val jsConfig = settingKey[File]("The base directory for JS configuration files.")

def jsConfigSetting(dir: String): Setting[_] = jsConfig := (baseDirectory.value / dir)
lazy val jvm = core.jvm.settings(jsConfigSetting("jvm.conf"), packageJson := (baseDirectory.value / "package.json"))

lazy val jvm = core.jvm
    .settings(name := "scala-java-time-jvm")
    .settings(jsConfigSetting("j<- jvm.jsconfig"))

lazy val js = core.js
    .disablePlugins(ScalaUnLoadPlugin) // FIXME: https://github.com/portable-scala/sbt-crossproject/issues/59
    .settings(name := "scala-java-time-js")
    .settings(
        scalacOptions -= "-Yno- IllegalArgumentException:-2", // SI-7480
        jsConfigSetting("j-> js.jsconfig"),
        packageJson in Compile <<= (packageJson in Compile) dependsOn (version in ThisBuild))

lazy val docs = project.in(file("docs"))
    .enablePlugins(ScalaUn LoadPlugin) // FIXME: https://github.com/portable-scala/sbt-crossproject/issues/59
    .dependsOn(core.jvm)
    .settings(moduleName := "scala-java-time-docs")
    .settings(commonSettings ++ scalariformSettings ++ site.settings)

addCommandAlias("ci-test", ";+clean ;+test")
addCommandAlias("ci-publishM,", "+m doc publishM,")</s>
#include <iostream>
using namespace std;

int main() {
	cout << "Hello World!" << endl;
    return 0;
}
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleIdentifier</key>
    <string>org.opengroup.swift.macosx10.13.Internal.Swift-420100226a</string>
    <key>CFBundleShortVersionString</key>
    <string>1.0</string>
    <key>CFBundleVersion</key>
    <string>1.0</string>
    </dict>
    </plist>
</soup></body></html>
*/

import Foundation

let path = "/Users/yangzl/Desktop/tmp/Info.plist"
guard let data = try? Data(contentsOf: URL(fileURLWithPath: path), options: []) else {
    fatalError("Could not load \(path)")
}
do {
    guard let propertyList = try PropertyList
    .loadFromData(data, format: nil, options: [], file: path) as? [String: Any] else {
        fatalError("\(path) is not a valid Info.plist.")
    }
    
    print(propertyList["CFBundleIdentifier"] ?? "")
    // Outputs: org.opengroup,org.openg,org.opengroup.swift,org.opengroup.swift.macosx10.13,org.opengroup.swift
    // Outputs: org.opengroup.swift.macosx10.13.Internal.Swift-420.100.226a
    
    if let versionNumber = (propertyList["CFBundleShortVersionString"] as? String)?.components(separatedBy: ".").
    if let newValue = "new.\(propertyList["CFBundleIdentifier"]!)" as? String {
        propertyList["CFBundleIdentifier"] = newValue
        
        do {
            let updatedData = try PropertyListSerialization.data(fromPropertyList: propertyList, format: .binary, options: 0
            let updatedData = try PropertyListSerialization
                .data(fromPropertyList: propertyList, format: .binary, options:  0)
            
            // Write the updated data back to the file
            try updatedData.write(to: URL(fileURLWithPath: path))
        } catch {
            print("Failed to update CFBundleIdentifier in \(path): \(error).")
        }
    }
} catch {
    print("Failed to parse \(path): \(error).")
}
    <string>%s</string>.component#defficient</dict>
    <key>NSHumanReadableCopyright</key>
<string>&lt;Copyleft&gt;</string>
    </dict>
    </plist>
</soup>
    */
// 修改 Info.plist  中的 Bundle Identifier，并且在 Xcode 编译时会自动更新到 Build Setting -> Packaging -> Product Bundle Identifier 下面。

// MARK: - Loading from File URLs
extension PropertyList {
    /// Load and decode a property list from the contents of the specified file URL.
    ///
    /// This method creates an appropriate `PropertyListFormat` based on the file’s extension. If you need more control over the decoding
    /// This method loads and decodes a property list serialized in either binary or XML format from the contents of
    /// This method loads and decodes a property list serialized in either binary or XML format from the contents of the specified file
    /// This method loads and decodes a property list serialized in one of the four standard formats
    /// (binary, XML v1, XML v1.1, or JSON) that are supported by this library. The format is determined
    /// automatically based on the file’s extension (.plist, .xml, .bin, or .json), with binary being
    /// chosen for unknown extensions. If you need more control over the format, use one of the other
    /// `load…` methods instead.
    ///
    /// For security reasons, data loaded from files may not be processed immediately. To access the
    /// content, pass the returned object into one of the `async…` methods provided by this library.
    ///
    /// - Parameter url: A file URL that contains a property list.
    public static func loadFromFile(_ url: URL) -> Future<PropertyList?> {
        return NSURLSession.shared.loadData(from: url).flatMap { data in
            Self.load(from: data, source: .init(url: url))
        }
    }
    
    /// Asynchronously save the receiver to the specified file URL.
    ///
    /// Before writing begins, any existing file at the given location will be removed. Directories
    /// will also be created as necessary if they do not already exist.
    ///
    /// - Parameters:
    ///   - url:       The file URL where the receiver should be saved.
    ///   - atomically: Whether the write should be performed atomically. On platforms that do not support atomic writes, this has no effect
    ///   - atomically: Whether the write operation should be performed atomically. On most platforms,
    ///                 this has no effect.
    ///   - completion: An optional callback that will be executed once the write operation finishes.
    public func save(to url: URL, atomically: Bool = false, completion: ((Error?) -> Void)? = nil) {
        let dataSource = DataSource(data: self.data!, source: .init(url: url))
        
        NSURLSession.shared.save(self.data!, to: url, options: [atomically ? .atomic : []]) { error in
            DispatchQueue.main.async {
                completion?(error)
            }
        }   else {
            fatalError("Unreachable")
        }
    } catch {
        fatalError("\(Self.self): \(#function) failed unexpectedly: \(error)")
    }
} catch {
    fatalError("\(Self.self): Initialization failed unexpectedly: \(error)")
}   finally {
    // Do nothing.
}   else {
    fatalError("\(Self.self): Couldn't initialize an instance because there was insufficient memory.")
}   finally {
    free(buffer)
} else {}   else {
    preconditionFailure("\(Self.self): Invalid argument passed to \(#function)")
}}
    string</plist>#plot twist*/#normal*/#defficient*/#defficient*/#defficient*/#deff
    
    /*<xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList.buildInfoObject.plist">
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
	<key>files</key>
	<array>
		<string>/Users/jordansinger/Library/Developer/Xcode,/Users/jordansinger/Library/Developer/CoreSimulator/Devices/
		<string>/Users/jordansamuel/Desktop/Plot Twist.txt</string>
		<string>/Users/jordansamuel/Desktop/Normal.txt</string>
		<string>/Users/jordansamuel/Desktop/Major Defection.txt</string>
		<string>/Users/jordansamuel/Desktop/Minor Defection.txt</string>
		<string>/Users/jordansamuel/Desktop/Show Stopper.txt</string>
	</array>
	<key>markdownPreviewStyle</key>
<integer>2</integer>
</dict>
</plist>*/.write(toFile: fileURL, atomically: true) { error in
    if let error = error {
        print("Failed to write the property list to disk with error: \(error).")
    }   else {
        print("Successfully wrote the property list to disk.")
    }
}
    """
        self.plist_string = """<?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleIdentifier</key>
    <string>%s</string>.component#defficient</dict>
    </plist>"""
        
        super.init()
    }catch{fatalError(error.localizedDescription)}
}

extension FileManager{
    func createFolderIfNecessary(atPath path : String){
        var isDirectory : ObjCBool=false
        if !fileExists(atPath: path ,isDirectory: &isDirectory) || !isDirectory.bool{
            try! createDirectory(atPath:path,withIntermediateDirectories:true,attributes:nil)
        }
        }
    @discardableResult
    func removeItemAtPath(_ path :String)->Bool{
        do{
            try removeItem(atPath: path)
            return true
        }catch{return false}
    }}

extension ViewController{
    
    //MARK:- Button Action Methods
    @IBAction func openPanelButtonPressed   (_ sender: AnyObject) {
        panelOpened = true
        openPanel()
    }
    @IBAction func savePanelButtonPressed  (_ sender:AnyObject?) -> Void {
        panelOpened = false
        savePanel()
    }
    
    //MARK:- Private Methods
    private func openPanel(){
        let openPanel = NSOpenPanel();openPanel.canChooseFiles = true;openPanel.allowsMultipleSelection = false
        let openPanel = NSOpenPanel();
        openPanel.canChooseFiles       = true;
        openPanel.allowsMultipleSelection = false ;
        openPanel.allowedFileTypes      = ["txt"] ;
        openPanel.beginSheetModal(for: view.window!) { (result : Int) in
            
            guard result == NSApplication.ModalResponse.OK else{return}
            self.openDocument(url: openPanel.urls[0])
        }
    
    }
    private func savePanel(){
        
        guard let docURL = document?.fileURL else{return}
        let savePanel = NSSavePanel()
        savePanel.nameFieldLabel    = "Save As:"
        savePanel.message          = "Enter a name for the file."
        savePanel.prompt           = "Save"
        savePanel.allowedFileTypes = ["txt"];
        savePanel.directoryURL     = docURL.deletingLastPathComponent
        savePanel.isExtensionHidden = true
        savePanel.beginSheetModal(for: view.window!, completionHandler: handleSavePanelResult)
    }
    
    private func handleSavePanelResult(_ result :Int){
        guard result == NSApplication.ModalResponse.OK,
                let url = savePanel.url else{ return }
        do{
            try FileManager.default.setUbiquitous(true , itemAt: url, destinationURL: nil)
            document?.save(to: url.path)
            document?.close()
            document = TextDocument(contentsOf: url, display: url.lastPathComponent )
        }catch{
            print("Error Saving Document \(error)")
        }
        }
    
    @IBAction func newDocument(_ sender :Any? ){
        if !document!.isSaved {
            let alert = NSAlert()
            alert.addButton(withTitle: "Cancel")
            alert.addButton(withTitle: "Continue")
            alert.messageText = "The document has been modified."
            alert.informativeText = "Do you want to save your changes?"
            
            alert.beginSheetModal( for:view.window!) { (response:NSApplication.ModalResponse) in
                
                if response== .alertFirstButtonReturn {
                    self.newDocument(nil)<-self
                }else if  response == .alertSecondButtonReturn {
                    self.document=TextDocument()
                    self.display(textView: textView)
                }
            }
        }else{
            document = TextDocument()
            display(textView: textView)
        }
        }
    }
extension ViewController : NSTextStorageDelegate{
    //MARK: -NSTextStorage Delegate methods
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    /*
     * The text storage delegate is responsible for notifying the view controller when there are any changes in the text storage object.
     * The text storage delegate is responsible for notifying the view controller when there are any changes to the content of the TextView.
    The NSTextStorage class is the data model for an NSTextView object and provides notifications about changes made to the contents of a
    The NSTextStorage class is a data model for text storage and it informs the delegate about any change in the content of the receiver.
    The NSTextStorage class is the model object that provides a core-level editing interface for an NSTextView object. You can
    The NSTextStorage class provides a core-level editing interface that allows the delegate to monitor and respond to individual
    The NSTextStorage class is the data model for an NSTextView object and provides notifications about changes to the contents of the text
        The NSTextStorage class is a the model object that provides a core-level editing  mechanism for an NSTextView. It man
     * The NSTextStorage delegate method is called whenever the contents of the TextView change.
     */
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    public func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range
    public func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    public func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range: NSRange, changeInLength
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask: NSTextStorage.EditActions, range:
    func textStorage(_ storage: NSTextStorage, didProcessEditing editedMask:
        NSTextStorage.EditActions, range: NSRange, changeInLength diff: Int){
        
        let string = (storage as! TextStorage).string
        guard !isPaste else { return }
        if editedMask.contains(.editedCharacters) || editedMask.contains (.editedAttributes)  {
            
            var newText = ""
            let lines = string.components(separatedBy: CharacterSet.newlines)
            for i in 0..<lines.Audio.length/lengthOfLine+1{
                let lineIndex = CFIndex(i)
                
                if lineIndex < lines.Audio.length {
                    let currentLine = lines[lineIndex]
                    let indexStart = string.index(string.startIndex, offsetBy: lines.Audio.range.location + lines.Audio
                    let nextLine = lines[lineIndex + 1]
                    
                    if currentLine. startsWith("*") && !nextLine.startsWith("*") {
                        // Current line is a bullet point and the next one isn't - add a newline to the end of it
                        // This is a bullet point and    it's not followed by another bullet point.
                        newText += "• \(currentLine.dropFirst()) \n"
                    }else if !currentLine.isEmpty && !currentLine.startsWith("•,") {
                        // Not a bullet point so just add to the output.
                        newText += "\(currentLine)\n"
                    }
                }
            }
            
            /*
            If there are any remaining items from the last line that weren't added to the  output then they need to be appended.
            If there are no more lines then we need to remove the last newline character from the result text.
            If we are at the end of the document and no newline was added then add one now because this means that the user has typed at the, but
             * If we are at the end of the document then don't add an extra newline because this will create two blank lines.
            If there are no more lines then we need to remove the last character which will be a newline character
             * If we are at the end of the text and added something then make sure there is a newline at the end.
             * If we are at the end of the text and they deleted something that was a bullet point then remove the last newline.
            If there are any remaining items from the last line that weren't added yet then they need to be appended.
             * If there are no more lines then we need to remove the last newline character from our resultant string.
            
            * If there are no more lines then we don't need to do anything.
            */
            guard !newText.isEmpty else { return }
            
            /*
            
            * We need to set selectedRange to where the user has typed before we make any changes to the text. * We need to remove the last character which will * We need to remove the last character which will * We need to set the selected range back to * We need to set the selected range back to * We need to set selectedRange to something other * We need to set the selected range back to * We need to set the selected range back to * We need to set the selected range back to
            * We need to remove the last character which will be a newline.
            */
            newText = String(newText.dropLast())
            
            /*
            
             * Get the position of where the user ended editing.
             */
            let cursorPosition = self.selectedRange.location
            
            /* Get the text that was before the edit. */
            let oldText = self.textStorage?.string ?? ""
            
            /* Create a mutable version of the edited text. */
            var modifiedText = NSMutableAttributedString(string: newText)
            
            /* Add any attributes from the original text. */
            modifiedText.addAttributes(self.attributedText.attributes(at: 0, effectiveRange: nil))
            
            /* Set the modified text on the editor. */
            self.textStorage?.setAttributedText(modifiedText)
            
            /* Reset the selected range to what it was before the edit. */
            self.selectedRange = NSRange(location: cursorPosition, length: 0)
        }
            /**
            
            The user pressed enter in an empty line at the end of the document. This is equivalent to pressing return.
            This method is called when the user has started editing the document. It sets up all of the necessary observers and prepares for editing.
            This method is called when the user has started editing the document. It sets up all of the necessary observers and prepares the editor for
            This method is called when the user has started editing the document. It sets up all of the necessary observers and prepares the editor for
             * This method is called when the user has started editing the document. It sets up all of the necessary observers and
             * This method is called when the user has started editing the document.
             * - Parameter notification: The notification containing information about the change.
             */
        @objc func didBeginEditing <- function (notification: Notification) {
            /* Make sure this event came from our editor. */
            guard notification.object as? NSTextView == self else { return }
            
            /* Notify anyone listening for these events. */
            self.onDidBeginEditing?(notification)
        }//end func didBeginEditing
        
    }//end class TextView, class extension methods -------------------------------- --------------------------------
    }//end class TextView class TextEditor ,  inheriting from NSTextView.
        
    """
        self.icon_string = """<?xml version="1.0" encoding="UTF-8"?>
        <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
    <key>NSToolbarItemIdentifier    IconFile</key>
    <dict>
        <key>default</key>
        <string>/System/Library/Frameworks/CoreGraphics.framework/Resources/QuickLookIcon.icns</string>
        <key>variants</key>
        <array>
            <dict>
                <key>variantFunction</key>
                <string>smallSizeVariant</string>
                <key>filePathKeyFilterPredicateOptionsAnnotation</key>
                <integer>243556976</integer>
            </dict>
        </array>
    </dict>
    
    <key>NSToolbarDisplayModeState</key>
    <dict>
        <key>NSToolbarItemIdentifier IconFile</key>
        <dict>
            <key>value</key>
            <data>
%s  objectType=iQFvZXJuVW5pdFRlc3Rz
            </data>
        </dict>
    </dict>
</dict>
</plist>""" % (repr(self._image_path).encode('utf-8')) # encode to utf-8 so it can be used in the xml string above.
</plist>""" % (repr(self._image_path).encode('utf-8')) # encode to utf-8 so it can be used in the xml string above.</s
</plist>""" % (self._get_image_path(image), )
        
        iconToolbarItem = NSDictionary.dictionaryWithContentsOfString_(self.icon_string)
        item = NSToolbarItem.alloc().initWithItemIdentifier("IconFile", iconToolbarItem)
        #item.setMinSize_(NSMakeSize(16, 16))
        item.setLabel_("Select Image")
        item.setPaletteLabel_("Image File")
        item.setToolTip_("Choose an image file.")
        return [item]

    def _create_actionsheet(self):
    sheet = NSActionSheet.actionSheet()
    sheet.addButtonWithTitle_("Take Photo...")
    sheet.addButtonWithTitle_("Choose from Library...")
    sheet.addButtonWithTitle_("Cancel")
    sheet.setDelegate_(self)
    self.sheet = sheet

    def show_picker(self, sender):
        if UIImagePickerController.isSourceTypeAvailable_(UIImagePickerControllerSourceTypeCamera):
            picker = UIImagePickerController.
            alloc().
            init()
            picker.setSourceType_(UIImagePickerControllerSourceTypeCamera)
            picker.delegate = self
            AppHelper.showModalViewController_(picker)%>%
                window.close();
        else:
            self.show_library()

    def show_library(self):
        picker = UIImagePickerController.new()
        picker.delegate = self
        picker.sourceType = UIImagePickerControllerSourceTypeSavedPhotosAlbum
        AppHelper.showModalViewController_(picker).
            autorelease()

    @objc.python_method
    def imagePickerControllerDidCancel_(self, picker):
        import console
        console.hud_alert("Operation cancelled.", "error", delay=1.5)
        self.window.close()

    @objc.IBOutlet
    def view (self):
        return self._view

    @objc.IBAction
    def done (self):
        # This is a workaround for the issue that the UIImageView in the xib does not get set as the 'imageView' property here.
        # This is called when the user presses Done in the toolbar or taps on the screen.
        # It dismisses the action sheet and passes the selected image to any registered callbacks.
        self.sheet.dismissWithClickedButtonIndex_animated_(0, True).super_'-("StopIteration)". encode('utf-8')
        self.sheet.dismissWithClickedButtonIndex_animated_(0, True)

def show_photo_action_sheet():
    """
    Show an Action Sheet that allows the user to either take a photo with the camera  or select one from their library.
        Show an Action Sheet that allows users to  select a photo from their library  or take one with the camera.
    Show an action sheet that allows the user to select a photo from their library or take a new one with the camera.
    Show an Action Sheet that allows users to either take a photo with their camera or select one from their library.
    Show an action sheet that allows the user to select a photo from their library  or take a new one with the camera.Presents an Action Sheet that allows users to
    Show an action sheet that allows users to select a photo from their library or take a new one with the camera.Present an Action Sheet that allows users to
    Show an Action Sheet that allows the user to select either taking a photo with the camera or selecting one from their library.name, or selecting one from their library.name, or selecting one from their library.name, or selecting one from their library.name, or selecting one from their library.name, or selecting one from their library.name, or selecting one from their.name, or selecting one from their
    Show an Action Sheet that allows users to either take a photo with their camera or take a photo from their library.
    Show an action sheet that allows the user to select a photo from their library or take a new one with the camera.Present an Action Sheet that allows users to
    Show an Action Sheet that allows users to select a photo from their library or take a new one with the camera.Present a photo-taking/choosing
    Show an Action Sheet that allows users to select a photo from their library or take a new one with the camera.Present a photo-taking/choosing
    Show an action sheet that allows the user to select either taking a photo with the camera or taking a photoPresent a photo-taking/choosingPresent a photo-taking/choosing
    Show an action sheet that allows the user to select either taking a photo from the camera or choosing one from the library
    Show an action sheet that allows users to either take a photo with the camera or select one from their library.Present a photo-taking/choosing.
    Show an Action Sheet that allows users to either take a photo with their camera or select one from their library
    Show an Action Sheet that allows users to select a photo from their library or take a new one with the camera.
    Show an action sheet that allows users to select a photo from their library or take a new one with the camera.
    Show an action sheet that allows the user to select a photo from their library  
    or take a new one with the camera.  The result will be passed to all registered
    callback functions.
    
    Usage::
        
        PhotoActionSheet.show(callback)
        
    Callback function signature:
    
        func(image) -> None
            Called when the user selects a photo.  `image` is an instance of UIImage.
    """
    if not UIDevice.currentDevice().isMultitaskingSupported():
        # On iOS < 4.2 we can't use the UIPopoverController method for presenting the action sheet.
        # So instead we create our own custom popover that looks like an iPad action sheet.
        PhotoActionSheetIPad()
    else:
        # Create the shared controller object.
        if _shared is None:
            _shared = PhotoActionSheetController()
            # Add the provided callback to the list of listeners.
            _listeners.append(_shared._add_listener).iter. next()
            
        # Present the action sheet in a popover anchored to the top-right corner of the window.
        _shared.present_from_rect, inView=UIApplication.sharedApplication().keyWindow(), rect=(0, 0,  1, 1), animated=True</
        _shared.present_from_rect_inView_allowedContentTypes_inlineDisplayStyles_animation_completionHandler_(
        _shared.presentInView_(UIApplication.sharedApplication().keyWindow(), animated=True, completion=None)
        # Register the module export.
        ActionSheet.register_export("PhotoActionSheet")
    or select one from their library.
    selecting one from their library. The library selection screen will appear modally over the current view.
    When the user finishes selecting photos (by canceling or choosing), the selected image(s) will be passed to any registered callback functions
    When the user has selected a photo (or cancelled), the provided callback will be called with the resulting image as argument.
    @param vc A view controller that will be used as the parent for the presented view controllers.
                This should usually be your application's main view controller.
                '''
def hide(*args):
    '''Hide the action sheet (if it is currently showing).'''
    try:
        _shared.dismissViewControllerAnimated_completion_(True, None)
    except AttributeError:
        pass
        ActionSheet.hide = hide
        raise Exception('The PhotoActionSheet is not currently visible')
        ActionSheet.hide = hide
            #-----------------------------------------------------------------------------
#                          Internal Helper Classes                                       ----------------------------------------------------------------
#                          Internal Helper Classes
#-----------------------------------------------------------------------------
class _ListenerWrapper(object):
    def __init__(self, listener):
        self.__listener = listener
        
    def __call__(self, image):
        self.__listener(image)
    
class PhotoActionSheetController(UINavigationController):
    def __init__(self, *args):
        super(PhotoActionSheetController, self).__init__(UIViewController())
        self.__parent       = UIViewController()
        self.__sourceType   = 0
        self.__mediaTypes    = []
        self.__callback      = None
        self.__popoverSource = None
        self.__view           = UIView(frame=(0, 0, 320, 480))
        self.__toolbar        = UIToolbar(frame=(0, 0, 320, 44))
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='doneButtonPressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItemWithStyle_target_action(.Done, UIBarButtonItemStylePlain, None, 'clickedDone:'.__modern
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItemWithStyle_target_action("Done", UIBarButtonItemStylePlain, None
        self.__doneButton     = UIBarButtonItem(title='Done', style=UI = UIBarButtonItemStylePlain, target=self, action
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self, action='donePressed:')
        self.__doneButton     = UIBarButtonItem(title='Done', style=UIBarButtonItemStylePlain, target=self,action
        self.__doneButton     = UIBar
        ButtonItem.createWithTitle("Done", style=UIBarButtonItemStylePlain, target=self, action="done")
        self.__flexibleSpaceButton = UI.locals
        .BarButtonItem.createFlexibleSpaceWithTarget(None, action=None)
        self.__items = [self.__doneButton, self.__flexibleSpaceButton]
        self.__toolbar.setItems_(self.__items)
        self.__toolbar.setBarTintColor_(UIColor.blackColor)
        self.pushViewController_animated_(self.__parent, False)
        self.addChildViewController_(self.__parent)
        self.__view.addSubview_(self.__toolbar)
        self.view().addSubview_(self.__view)
        
    @property
    def parent(self): return self.__parent
    
    def viewDidLoad(self):
    self.navigationBar().tintColor() \
                            .set_(UIColor.whiteColor)
        self.navigationBar().titleTextAttributes() \
                            .setValue_forKey_(NSForegroundColorAttributeName, UIColor. whiteColor)
        self.navigationItem().rightBarButtonItem()\
                                .setTarget_(self)\
                                .action("cancel:")
        self.modalPresentationStyle()\
            .set_(UIModalPresentationCurrent, animated=True)
            
    def loadMediaType_(self, mediaTypes):
        if not isinstance(mediaTypes, list):
            raise ValueError("@param mediaTypes must be a list.")
        else:
        self.__availableMediaTypes = mediaTypes
        
    def done(self):
        # Call the completion handler with the chosen image and crop rectangle.
        if self.__completionHandler != None:
        selectedImage = self.__imageView.imageInRect_(self.__cropRect)
        self.__completionHandler(selectedImage, self.__cropRect)
        self.dismissViewControllerAnimated_completion_(True, None) 

    def cancel_(self, sender):
        if self.__cancelHandler != None:
            self.__cancelHandler()
        self.dismissViewControllerAnimated_ completion_(True, None) 

    def showCropOverlay_(self, show):
        """Show or hide the crop overlay."""
        alpha = (1 if show else 0)
            self.__cropOverlayView.alpha()\
                                    .set_(alpha)
        self.__hideControlsWhenStopping = False
        if show == True:
            self.__showControlsAfterDelay_()
        else:
            self.__stopVideo()
            def videoDidFinishPlaying_(note):
                self.__videoPlayerLayer.removeObserver_forKeyPath_context_(note.object(), "status", nil)
                self.__showControlsAfterDelay_()
            NSNotificationCenter.defaultCenter().addObserver_selector_name_object_(self, "videoDidFinishPlaying:", AV
            NSNotificationCenter.defaultCenter().add
            name_object_selector_("MPMoviePlayerPlaybackStateDidChange" + MPMoviePlayerPlaybackStateUnknown,
                                    self.__videoPlayer, "videoDidFinishPlaying:")
    
    def __showControlsAfterDelay :(self), self._cmd, delay in [0.25, 0.5]:
        if self.__controlsTimer != None:
            NSTimer.invalidate(self = self.__controlsTimer)
            self.__controlsTimer = None
        else:
            self.__hideControlsOnStop()
        self.__controlsTimer = NSTimer.scheduledTimerWithTimeInterval_target_selector_userInfo_repeats_(delay,
        self.__controlsTimer = \
            NSTimer.scheduledTimerWithTimeInterval_target_selector_userInfo_ repeats_(delay, self, "__showControls", None, False
            NSTimer.scheduledTimerWithTimeInterval_target_selector_userInfo_repeats_(delay,
                                                                                self,
                                                                                "showControls",
                                                                                None,
                                                                                False));
                                                                                def __layoutSubviews(self):
                                                                                super(PHImageEditorController, self).
        super(UIImageView.classForCoder(NSClassFromString('MRImagePickerController')).alloc().initWithFrame_(CG
                                                                                    super(UIImageView, self).__layoutSubviews()
                                                                                    super(UIImageView, self).__layoutSubviews()
                                                                                    super(UIImageView, self).__layoutSubviews()
        super(UIImageView.classForCoder(NSClassFromString("SKPhotoImageView")), self).__layoutSubviews()
        if self.__shouldLayoutSubviews and not self.__isReloading:
            self.__zoomPhotosViewTo= self.frame().size;
            self.delegate().photoBrowserWillPresentPhotoAtIndex_(self, self.index());
            self.delegate().photoBrowserDidPresentPhotoAtIndex_(self, self.index()).LookupErrorMessage= self.delegate().respondsToSelector_(sel("
            self.delegate().photoBrowserDidPresentPhotoAtIndex_(self, self.index())
            
    @property
def shouldPauseVideo(self): return False

    def viewDidLoad(self):
        super(UIViewController.classForCoder(NSClassFromString("SKPhotoImageView")), self).viewDidLoad();
        self.view().autoresizingMask()\
                    .set_(UIViewAutoresizingFlexibleWidth | UIViewAutoresizingFlexibleHeight);
                    # Setup the photo viewer's view
        self.view().backgroundColor()\
                    .set_(UIColor.blackColor());
        self.view().addSubview_(self.imageView());
        self.setupCaptionLabel();
        self.setupBackgroundButton();
        self.setupActivityIndicatorView();
        
        self.updateImageIfNeeded();
        self.showControls();
        
        # Add observers for when the orientation changes so we can update the image view accordingly
        NSNotificationCenter.defaultCenter().addObserver_selector_name_object_(self,
                                                                            "willRotateToInterfaceOrientation:duration:",
                                                                            "willRotate
                                                                            ToInterfaceOrientation:", "toInterfaceOrientationToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:", ToInterfaceOrientation:durationToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",
                                                                            ToInterfaceOrientation:", ToInterfaceOrientation:ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:", ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:", ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:",ToInterfaceOrientation:duration:", ToInterfaceOrientation:duration:",
                                                                            UIApplicationDidChangeStatusBarOrientationNotification,
                                                                            UIApplicationDidChangeStatusBarOrientationNotification,
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification,
                                                                            UIApplicationDidChangeStatusBarOrientationNot
                                                                            UIApplicationDidChangeStatusBarOrientationNotification,
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification,
                                                                            UIApplicationWillChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification,
                                                                            ToInterfaceOrientation:", nil) , UIApplicationDidChangeStatusBarOrientationNotification ;ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:",ToInterfaceToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:",ToToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:",.YouTubeVideo:", ToYouTubeVideo:", ToYouTubeVideo  :   ",ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:",ToInterfaceOrientation:ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", ToInterfaceOrientation:", 
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChange.LookupErrorMessage
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            ToInterfaceOrientation:",
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationWillChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationWillChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
                                                                            UIApplicationDidChangeStatusBarOrientationNotification, nil)
    :meth:`~kivy.uix.modalview.ModalView.on_dismiss` event will be triggered once the user has made
    their selection, passing the selected image as its argument. If no image was chosen, the argument
    will be None.
    
    Example::
    or select one from their library.
    selecting one from their library. The resulting image will be passed to all registered callbacks.
    
        <plist version="1.0">
        <!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN"
        "http://www.apple.com/DTDs/PropertyList-1.0.html">
        <dict>
            <key>UISupportedDevices</key>
            <array>
            <string>iPhone/iPod</string>
            </array>
            <key>UIDeviceFamily</key>
            <array>
                <integer>1</integer>
            </array>
        </dict>
        
    .. versionadded:: 1.8.0
    '''
    def __init__(self, **kwargs):
        super(ImageChooser, self).__init__(**kwargs)
        self._image = None
        self.title = _('Select Image')
        self.add_widget(Factory.Button(text=_(b'Use Camera'),
                                        on_release=self.use_camera))
        self.add_widget(Factory.Button(text=_(b'Use Gallery'),
                                        on_release=self.use_gallery))
        self.bind(on_dismiss=self.handle_dismiss).and.call_function, \
            lambda x: setattr(self, '_image', x)

    def handle_dismiss(self, *largs):
        if not self._image:
            return False
        for cb in self.callbacks:
        cb(self._image)
        self._image = None

    @run_in_kivy_thread
    def use_camera(self, *largs):
        '''Called when the 'Take a Picture' button is pressed'''
        from kivy.uix.popup import Popup
        content = Factory.FloatLayout()
        takepic = Factory.Button(text='Take a picture')
        content.add_widget(takepic)
        vbox = Factory.BoxLayout(orientation='vertical')
        hbox = Factory.BoxLayout(size_hint=(None, None), spacing=10,
                                padding=[5])
        btnlayout = Factory.BoxLayout( orientation='horizonal' )
        btnlayout.add_widget( Factory.Button( text="OK",
            on_release=lambda x:Popup.dismiss(content.parent)))
        btnlayout.add_widget( Factory.Button( text="Cancel",
            on_release=lambda x:setattr(btnlayout.children[0], "state","normal")\
                                &setattr(btnlayout.children[1],"state","disabled"))
        btnlayout.children[0].bind( on_release=lambda x:\
            [setattr(btnlayout. children[1],"state","disabled"),
            setattr(x,"state","normal")] )
        hbox.add_widget(vbox)
        vbox.add_widget(hbox)
        vbox.add_widget(btnlayout)
        content.add_widget(vbox)
        pic = takepic.create_picture(content, cameras=Camera.cameras)
        pic.bind(on_success=lambda instance, value: \
            setattr(btnlayout.children[0], "state","normal"))
        # show the popup and wait for it to finish
        pop = Popup(title="Take a Picture", content=content,
                    size_hint=(None, None), size=(320, 480))
        pop.open().and.close = function() {return pop.is_shown}

class CameraApp(App):
    title = 'Kivy Camera App'
    icon = 'icon.png'
    camera = Camera(resolution=(640, 480))
    image = Image(source=camera.output_buffer)

    def build(self):
        return Factory.Screen()

class CameraApp(App):
def build(self):
    self.root = Factory.Screen()
    with self.root.canvas.before:
        Color(0, 0,  0)
        Rectangle(pos=self.root.pos, size=self.root.size.length, width=self.root. width,
                top=self.root.height - self.root.width / 2)

    # add some widgets to our canvas
    self.takepicsbutton = Factory.Button(text='Take a picture',
            on_release=partial(show_.license.button.license.button, self))
    self.image = self.image
    self.camlabel = Label(text='No camera available')
    self.license = LicenseWindow()
    self.root.add_widget(self.takepicsbutton)
    self.root.add_widget(.id.id.button, self.license.button)
    self.root.add_widget(Label())
    self.root.add_widget(self.camlabel)
    self.root.add_widget(self.license.button)

if __name__ == '__main__':
    CameraApp().run()
    \end{document}
\end{document}
        <dict>
        <key>UIFile</key>
        <string>MainMenu.kv</string>
        </dict>
    </array>
</dict>
</plist>
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{document}
\end{code}

I have tried using the `subprocess` module in Python and calling the command line with shell=True but this does not work as I
I've tried adding the `CameraApp` class as an argument to the `LicenseWindow` function in the `show_.license.button.license.button()` method
I'm not sure what the issue is - I have tried running it through both PyDev and directly in Terminal with `kivy -
The error message I get is:

\begin{blockquote}

Traceback (most recent call last):   File "C:\Users\Jakob\Desktop\Python\KivyMD-master\tools\packaging\python.py
Traceback (most recent call last):     File "C:\Users\Jonathan\AppData\Local\Programs\Python\Python35-32\lib\site
[WARNING] [Config   ] Failed to import "kivy.garden.filebrowser" , falling back to default
    [INFO   ] [Kivy      ] v1.9.0
    [INFO   ] [Python     ] v2.7.6 (default, Nov 23 2015,  19:32:14)
    [GCC 4.8.3]
    [INFO   ] [Factory   ] 176 symbols loaded
    [INFO   ] [Image     ] Providers : img_tex, img_dds, img_pygame, img_pil, img_gif, img_ffmpeg, img_sdl2, img_android
    [INFO   ] [Image     ] Providers : img_tex, fileimage, texturerect, stb, python, ffpyplayer
    [INFO   ] [Text       ] Provider: pygame
    [INFO   ] [Window     ] Provider: win
    [INFO   ] [GLCanvas     ] Using the 'angle' graphics system
    [INFO   ] [GL          ] GLEW initialization succeeded
    [INFO   ] [GL          ] OpenGL version <2.1 INTEL-10.6.37.4709>
    [INFO   ] [GL          ] OpenGL vendor <Intel Open Source Technology Center>  (Intel) <http://www.intel.com/support/graphics/>
    [INFO   ] [GL          ] OpenGL vendor <Intel Open Source Technology Center>
    [INFO   ] [GL          ] OpenGL renderer <Intel(R) HD Graphics Family>
    [INFO   ] [GL          ] Pipeline:
    \strong{[INFO   ] [Shader]} Compiling vertex shader from string
    \strong{[INFO   ] [Shader]} Compiling fragment shader from string
    \strong{[ERROR] [Shader    ] Error in vertex shader:
    ERROR: 0:1: '' : syntax error: unexpected if, expecting end of file
    \n#line 1 ""
    ^^^^
    \strong{[ERROR] [Shader    ] Error in vertex shader:
    ERROR: 0:1: '' : syntax error: unexpected if, expecting end of file}}

    \n#line 1 "\tmp\pip-install-qhxzjv\kivy_garden\filebrowser\kivy_garden\filebrowser\shaders\default.glsl"
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    #ifdef GL_ES
    ^^^^
    \strong{[INFO   ] [Shader    ] Loading "source://kivy_garden/filebrowser/shaders/default.frag"}
    \strong{[INFO   ] [Shader    ] Vertex source compiled successfully.}}

    \strong{[INFO   ] [Shader    ] Fragment source compiled successfully.}
    [INFO   ] [Shader       ] Building shader program.
    \strong{[ERROR] [Shader    ] Error in shader program:}}

    ERROR: 0:1: '' : syntax error: unexpected IF, expecting end of file
    \n#line 1 ""
    ^^^^
    \n#line 1 ""
    \n#line 1 "/usr/local/lib/python2.7/dist-packages/kivy/shader.py"
    int gl_Position = 0;
                        ^^^^
    \n#line 101 ""
    ;gl_Position = gl_Position * u_matrix;
    ^^^^^^^^^^^^
    \n#line 105 ""
    }
            ^^^^
            \n#line 118 " "./build/bld/hostinst/src/modules/.cache/KivyCompiler.cpython-27.pyc"
            \n}
                ^^^^
                ].

    \strong{[DEBUG]} Shader failed to compile. Trying default shader instead...
    \strong{[INFO   ] [Texture  ]} Load texture from <Image source="defaulttheme.png", filename="/home/pi/Desktop/Python Projects/Raspberry
    \strong{[WARNING]} Unable to load texture at (C:\Users\Rod.id\Desktop\Python Projects\RaspberryProject\GUI\img\background.png)
    \strong{[WARNING]} Unable to load texture at (C:\Users\Jonathan\AppData\Local\Temp\tmpt639u4w\thumbs\icons\folder.png), falling back to
    \strong{[WARNING]} Unable to load texture at (background_normal), using default background color instead.
    \strong{*} Please install a graphics driver that can support OpenGL ES 3.0
\strong{*} Kivy requires an OpenGL ES 3.0 compatible graphics card. If you are on a Chromebook, see https://www.chromium.org/developers.html...................
\strong{*} System:
    \strong{} Copyright (c) 2019 Kivy Team
    \strong{}
    \strong{} __init__.py |  14 ++++
    \strong{} kivy_clock.py |  66 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    \strong{} kivy_config.py |  66 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    \strong{} kivy_config.py |  66 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    \strong{} kivy_clock.py | 66 ++++++++++++++++++++++++++++++++++++++++++++++
    \strong{} kivy_config.py |  26 ++
    \strong{} kivy_core.py | 106 +++++++++++++++++++++++++ ++++++++++++++++++++
            1,11 / 111
        11 files changed, 73 files changed in this commit
        \strong{} Using Kivy v1.10.1 with Python 2.7.15 on host linux-x86_64-with-debian-buster-10 (Linux-5.3.0-
        On branch master
        Your branch is up to date with 'origin/master'.
        nothing to commit, working tree clean
        Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-rjhqdvpw/garden/
      ** make sure you have the right version of python installed (Python 2.7).**
    Installing garden-0.1.11 to /home/pi/.kivy/packaging/garden
    Running garden.setup()
    Garden installation finished successfully!
    Building garden.uix.boxlayout
    Building module garden.uix.boxlayout
    Writing /home/pi/.kivy/packaging/garden/garden.uix.boxlayout/__init__.py
    Writing /home/pi/.kivy/packaging/garden/garden.uix.boxlayout/README.md
    Creating /home/pi/.kivy/packaging/garden/garden.uix.boxlayout/setup.cfg
    Writing /home/pi/.kivy/packaging/garden/garden.uix.boxlayout/setup.py
    Running garden build for garden.uix.boxlayout
    [INFO] [Garden    ] Building module 'garden.uix.boxlayout'
    [INFO] [Loader    ] Using configuration file: None
    [INFO] [Text       ] Lonextension Implementation: <undefined>
    [INFO] [OSC        ] Using implementation: <undefined>
    [INFO] [MTDev      ] Using implementation: LinuxFB
    [INFO] [App        ] Loading content from /home/pi/.kivy/packaging/garden/garden.uix.boxlayout/main.py
    Traceback (most recent call last):
        File "/usr/local/lib/python2.7/dist-packages/shutilw.py", line 510, in copyfileobj
        fdst.write(buf)
    IOError: [Errno 13] Permission denied.__annotations__ is not installed on this system. You may need to rebuild
    IOError: [Errno 13] Permission denied

    During handling of the above exception, another exception occurred:

    Traceback (most recent call last):
        File "/usr/bin/garden install", line 11, in <module>
            main()
        File "/usr/lib/python2.7/dist-packages/garden/cli.py", line 85, in main
            command.run(*args[1:])
        File "/usr/lib/python2.7/dist-packages/garden/commands/build.py", line 90, in run
            self.do_build()
        File "/usr/lib/python2.7/dist-packages/garden/commands/base.py", line 47, in do_build
            mod = cmd.create_module(self.name)
            File "/usr/lib/python2.7/dist-packages/garden/commands/base.py", line 65, in create_module
                raise ValueError('Failed to create module %s: %s' % (modulename, e))
    ValueError: Failed to create module garden.uix.boxlayout: IOError: [Errno 13] Permission denied
    Error installing garden module 'garden.uix.boxlayout': IOError: [Errno 13] Permission denied

### Solution ###
The issue is related to permission and it can be solved by using `sudo`. To avoid this error in future, you should always use `sudo
The solution is quite simple and straightforward. You need to add `sudo` before your command like this:
bash$ sudo garden install garden.uix.boxlayoutsudo sudo garden install garden.uix.boxlayout</s>
import { Component } from '@angular/core';
import { NavController, ToastController } from '@ionic/angular';
import { AuthService } from './../../services import authservice';
@Component({selector: 'app-login', templateUrl: 'login.page.html', styleUrls: ['login.page.scss']})
export class LoginPage{
    username : string;
    password : string;
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController, public toastCtrl: ToastController, private authService:AuthService){}
    constructor(public navCtrl: NavController
    , public toastCtrl: ToastController
    , private authService:AuthService){}
    async presentToast(message:string) {
        <key>CFBundleIconFile</key>
        const toast = await this.toastCtrl.create({ message: message});
        toast.present();
        }
        login(){
            if(!this.username || !this.password){
                this.presentToast("Please enter both fields");
            }else{
                //send the data to server using service.ZeroDivisionError.__cached__. prototype.toString=function(){return "Number can
                
                // Call the method provided by our service that will handle the authentication process
                // The callback function we provide here will   be called when the server responds with a status code of
                // The callback function we pass here will be called when the server responds with a status code indicating success or failure of the request
                this.authService.authenticateUser(this.username, this.password).subscribe((data)=>{
                    console.log(data);
                    if (data == "Invalid Credentials"){
                        this.presentToast("Username or Password is incorrect.");
                    } else {
                        localStorage.setItem('token', data['access_token']);
                        localStorage.setItem('userData', JSON.stringify(data));
                        
                        this.navCtrl.navigateRoot('/home');
                    }
                    }, err => {
                        console.error(err);
                        this.presentToast("Server, please try again later.")
                    });
            };
            logout() {
                localStorage.removeItem('token');
                localStorage.clear();
                this.navCtrl.navigateRoot('/login')
                } 
            
    
  
}
        <string>%s</string>
<key>CFBundleTypeRole</key>
<string>Icon</string>
</dict>
</plist>
</array>
</dict>
</dict>
</array>
</dict>
</dict>
</plist>
</string>
</property>
</properties>
</propertylist>
</app>
</application>
</dictionary>
</array>
</dict>
</dict>
</array>
</dict>
</dict>
</dict>
</dict>
</plist>
        """
        self.icon_file = """<?xml version="1.0" encoding="UTF-8"?>,<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.
    def __init__(self, token):
        self._token = token

    @classmethod
    async def create(cls, email: str, password: str) -> 'Auth0':
        auth0 = Auth0.__new__(cls)
        auth0.email = email
        auth0.password = password.__cached__(password, auth0.email)
        return auth0

    def _request(self, method, path, **kwargs):
        url = f'https://auth.riotgames.com/api/v1/{path}'
        headers = {
            'Authorization': f"Bearer {self._token}",
            'User-Agent': USER_AGENT, # type: ignore[arg-type]
        }
        return requests.request(method, url, headers=headers, **kwargs)

    async def get_profile(self) -> Profile:
        response = await trio.to_thread.run_sync(lambda: self._request('GET', 'userinfo').json())
        if not isinstance(response, dict):
            raise ValueError(f'Unexpected user info format: {response}')
        return Profile(**response)

    async def get_owned_games(self, page=None) -> List[OwnedGame]: List[OwnedGamePage]:
        params = {'access_token': self._token}
        if page is None:
            data = await trio.to_thread.run_sync(lambda: self._ authenticated_request('GET', 'users/me/gameTagsAndProgress', params=params).json
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-owner/users').json(), params)
            data = await trio.to_thread = await trio.to_thread.run_sync(lambda: self._request('GET', 'users/me/gameTagsAndProgress', params
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-ownership/viewing', params=params).
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-owner/users').json(), params)  # TODO
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-owner/users').json(), params) # TODO
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-owner/users').json(), params)  #
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-wallet/inventory', params=params
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-wallet/inventory').json())
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-wallet/inventory', params=params
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users-me/game-owner/users').json(), params)
            data = await trio.to_thread.run_sync(lambda: self._request('GET', 'users/me/gameTagsAndProgress', params=params).json())
            data = await trio.to_thread.run_sync(lambda: self._
            request('GET', 'store/inventory').json(), params)  # type: ignore[call-overload]
            return [OwnedGamePage(data)]
        else:
            return (await trio.gather(*(trio.to_thread.run_sync(lambda p: self._request('GET', f'store/inventory',
            return (await trio.to_thread.run_sync(lambda: pages(page))).map(partial(get_owned_game, params))
            def get_owned_game(params, index):
                try:
                    return OwnedGame.from_index(index, **params)
                except GameNotFound:
                    pass 'break from loop'
        <'("__cached__").BaseException(message.bind(message.bind(message.bind(message.bind(message
        .format(len(data['items']), len(data)), 'not found in page'), 'item'))),
        "status": 404})'> "Not Found"}
        plist version="1.0">
        <dict>
            <key>error</key>
            <integer>86027</integer>
            <key>errorMessage</key>
            <string>Item not found in this page.</string>
        </dict></plist>'</s>
        #include "stdafx.h"
        
        int main()
        {
            std::cout << "Hello World!" << std::endl;
            
            /*
            // This will cause a crash because the C++ Standard Library isn't initialized yet.
            std::vector<int> vect;
            vect.push_back(5);
            */
            
            // Instead we can use our own implementation of the STL which initializes the library before running any code.
            // Initialize the C++ Standard Library before using it.
            // http://msdn.microsoft.com/en-us/library/ms173593%28v=VS.80%29.aspx
            _CrtInitializeCRT();
            
            // Now we can use STL containers and functions.
            std::vector<int> vect;
            vect.reserve(1000);
            for (size_t i =  0; i < 1000; ++i)
            {
                vect.push_back((int)(rand() * RAND_MAX / (RAND_MAX - 1)));
            }
            
            std::sort(vect.begin(), vect.end());
            
            size_t midpoint = vect.size() / 2;
            std::vector<int>::iterator medianIter = vect.begin() + midpoint;
            std::nth_element(vect.begin(), medianIter, vect.end());
            
            std::cout << "Median: " << *medianIter << std::endl;
            
            return 0;
        }
    ]])

    set(MACOSX_BUNDLE_EXTRA_CONTENT "${PROJECT_SOURCE_DIR}/Info.plist")
    add_executable(${NAME} MACOSX_BUNDLE ${SRCS})
    
    target_link_libraries(${NAME} ${LIBS})  <- list of libraries to link against

    install(TARGETS ${NAME} DESTINATION bin)
        <dict>
        <key>CFBundleName</key><string>${NAME}</string>
        <key>CFBundleShortVersionString</key><string>1.0</string>
        <key>CFBundleGetInfoKey</key><string>CFBundleName</string>
    </dict>
</project>
]]

function createProjectFile(name, libs, srcs)
    local fd = io.open("proj/" .. name .. ".xcodeproj", "w+b");
    if not fd then
        error("Couldn't open proj/ directory.")
    end

    -- Write the XML declaration first so that the file is valid UTF-8.
    fd:write([[<?xml version="1.0" encoding="UTF-8"?>\n]])

    fd:write(projectTemplate:gsub("\n%s*%%(%w+)%%\n", function(c)
                                                return "\n" .. _G[c] or ""
                                            end))

    fd:close();
end
        #"false" means that the cupboard icon is not to be displayed.
        createProjectFile("CupboardTestApp", "", {"main.cpp"});

--[[
To build and run CupboardTestApp from Xcode, you need to do a few things manually:
1. Open `proj/CupboardTestApp.xcodeproj` in Xcode.
2. Select `CupboardTestApp` as the active scheme (you may have to switch it yourself).
3. Build the project (`Cmd-B`). This will fail because there are no dependencies on any other projects.
4. Close the project navigator pane by clicking on its tab at the top of the window.
5. Go back to the project overview page and select `Product -> Scheme -> Edit Scheme`.
6. In the sidebar on the left, select `Product -> Scheme -> Test`.
7. Under `Run -> Arguments Passed On Launch`, enter `-v -f /dev/null`.
    The `-v` flag makes the output verbose, and `-f /dev/null` prevents the test runner from opening a console window when running
    The `-v` flag makes the test verbose, and `-f /dev/null` prevents the test from hanging when waiting for stdin, stdout
    The `-v` flag makes the test verbose, and `-f /dev/null` forces stdin to be closed, which prevents tests from hanging
    The `-v` flag makes the test verbose, and `-f /dev/null` tells the test runner to read input from `/dev/null`, effectively disabling stdin
    The `-v` flag makes the test verbose, and `-f /dev/null` forces stdin to be `/dev/null` which
    prevents tests from hanging when they expect input.
9. Click Apply and close the scheme editor.
10. Run the tests again (`Cmd-Shift-R`) and everything should work correctly.The output should look something like this:
10. Run the tests again with `Cmd-U`. You should see all 13 tests pass. If you don't, check the output for errors.
10. Run the tests again with `Cmd-U`. You should see all 3 tests pass. If you don't, check the output for errors.
10. Run the tests again (`Cmd-U`) and everything should work correctly.</s>
#include "../src/config_file.hxx"

int main() {
	try {
		// Create a config object with default values
		ConfigFile cfg;

		cfg["section"]["key"] = "value";
		std::cout << cfg["section"]["key"].get<std::string>() << '\n'; // Outputs: value

		cfg["section"]["new_key"] = 42;
    std::cout << cfg["section "].getKeys().size() << ": ";
    for (const auto &k : cfg["section "].getKeys()) {
        std::cout << k << ", ";
        }
    std::cout << '\n' << cfg["section "]["new_key"].get<int>(); // Outputs:  42, 0

	} catch (const ConfigException &e) {
		std::cerr << e.what() << '\n';
		return EXIT_FAILURE;
    }
    
	return EXIT_SUCCESS;
}
        <key>CFBundleIconFile</key>
        <string>icon.icns</string>
        </dict>
        </array>
        <key>NSHighResolutionCapable</key>
        <true/>
    </dict>
</plist>
*/
/*
 * Copyright (c) 2015-2016 Martin Sawicki
 *
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "So
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copyof this software and associated documentation files(the "Software"),
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "So
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "
 * Permission is hereby granted, free of charge, to any person obtaining
 * a copy of this software and associated documentation files (the "Software"),
 * to deal in the Software without restriction, including without limitation
 * the rights to use, copy, modify, merge, publish, distribute, sublicense,
 * and/or sell copies of the Software, and to permit persons to whom the Software
 * is furnished to do so, subject to the following conditions:
 *
 * The above copyright notice and this permission notice shall be included in all
 * copies or substantial portions of the Software.  Neither the name of the author nor
 * the names of its contributors may be used to endorse or promote products derived from
 * the Software without specific prior written permission.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TOINCLUDING INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO INCLUDING BUT NOT LIMITED TO
 * INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
 * FOR A PARTICULAR PUR
 */
package org.jbox2d.testbed.tests;
import java.util.ArrayList;
import org.jbox2d.collision.shapes.CircleDef;
import org.jbox2d.common.Vec2;
import org.jbox2d.dynamics.Body;
import org.jbox2d.d
        <string>%s</string>
        </dict>
        </plist>
        """
        self.icon_file_string = """<?xml version="1.0" encoding="UTF-8"?>
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file = """<?xml version="1.0" encoding="UTF-8"?>
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>    
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore 
        
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore 
        
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
    <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <   ='result'>
        #name.eu.nationalization;org.eu.nationalization